In [1]:
import pandas as pd
import os

In [3]:
ruta_looks = os.path.join("Datos alumnos","Originales", "Datos looks")

df_products = pd.read_csv(os.path.join(ruta_looks, "product_2.csv"), sep=';')
df_brands = pd.read_csv(os.path.join(ruta_looks, "brand.csv"), sep=',')
df_features_value = pd.read_csv(os.path.join(ruta_looks, "feature_value.csv"), sep=',')
df_feature = pd.read_csv(os.path.join(ruta_looks, "feature.csv"), sep=',')
df_prod_feat = pd.read_csv(os.path.join(ruta_looks, "product_feature_value.csv"), sep=',')
df_prod_feat_qual = pd.read_csv(os.path.join(ruta_looks, "product_feature_value_qualifier.csv"), sep=',')
df_prod_variant = pd.read_csv(os.path.join(ruta_looks, "product_variant.csv"), sep=',')
df_season = pd.read_csv(os.path.join(ruta_looks, "season.csv"), sep=',')
df_size = pd.read_csv(os.path.join(ruta_looks, "size.csv"), sep=',')
df_color = pd.read_csv(os.path.join(ruta_looks, "color.csv"), sep=',')
df_feature_value_family = pd.read_csv(os.path.join(ruta_looks, "feature_value_family.csv"), sep=',')

## Juntamos csvs

In [5]:
# PASO 1: LIMPIEZA INICIAL DE NOMBRES
df_prod_clean = df_products.rename(columns={
    'id': 'product_id', 
    'title': 'product_name',
    'brand_id': 'brand_id'
})

df_variant_clean = df_prod_variant.rename(columns={
    'id': 'variant_id',
    'product_id': 'product_id',
    'color_id': 'color_id',
    'size_id': 'size_id',
    'season': 'season_code'
})

df_brand_clean = df_brands.rename(columns={'id': 'brand_id', 'name': 'brand_name'})
df_color_clean = df_color.rename(columns={'id': 'color_id', 'name': 'color_name'})
df_size_clean  = df_size.rename(columns={'id': 'size_id', 'europe': 'size_europe'})
df_season_clean = df_season.rename(columns={'season': 'season_code'})

df_fv_clean = df_features_value.rename(columns={'id': 'feature_value_id', 'value': 'feature_value_name'})
df_f_clean = df_feature.rename(columns={'id': 'feature_id', 'name': 'feature_type_name'})


In [6]:

# PASO 2: TABLA MAESTRA DE VARIANTES
df_master = df_variant_clean.merge(df_prod_clean, on='product_id', how='left')
df_master = df_master.merge(df_brand_clean, on='brand_id', how='left')
df_master = df_master.merge(df_color_clean, on='color_id', how='left')
df_master = df_master.merge(df_size_clean, on='size_id', how='left')
df_master = df_master.merge(df_season_clean, on='season_code', how='left')

df_master['brand_name'] = (
    df_master
    .groupby('product_id')['brand_name']
    .transform(lambda x: x.ffill().bfill())
)


# PASO 3: PREPARAR FEATURES
df_features_full = df_prod_feat.rename(columns={'id': 'pfv_id'})
df_features_full = df_features_full.merge(df_fv_clean, on='feature_value_id', how='left')
df_features_full = df_features_full.merge(df_f_clean, on='feature_id', how='left')

df_features_simple = df_features_full[
    ['product_id', 'feature_type_name', 'feature_value_name']
]


C:\Users\marke\AppData\Local\Temp\ipykernel_19032\2707740007.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .transform(lambda x: x.ffill().bfill())


In [7]:
# PASO 3: PREPARAR FEATURES
df_features_full = df_prod_feat.rename(columns={'id': 'pfv_id'})
df_features_full = df_features_full.merge(df_fv_clean, on='feature_value_id', how='left')
df_features_full = df_features_full.merge(df_f_clean, on='feature_id', how='left')

df_features_simple = df_features_full[
    ['product_id', 'feature_type_name', 'feature_value_name']
]


In [10]:
# PASO 4: MERGE FINAL
df_final_big = df_master.merge(
    df_features_simple,
    on='product_id',
    how='left'
)


# VERIFICACIÓN
print(f"Filas originales (Variantes): {len(df_master)}")
print(f"Filas finales (Variantes x Features): {len(df_final_big)}")
print("-" * 30)

example_id = df_final_big['variant_id'].iloc[0]
cols_to_show = [
    'variant_id',
    'product_name',
    'size_europe',
    'feature_type_name',
    'feature_value_name'
]


Filas originales (Variantes): 65226
Filas finales (Variantes x Features): 662003
------------------------------


In [11]:
df_final_big


,variant_id,barcode,color_id,legacy_id_x,product_id,season_code,size_id,stock,publishable,Unnamed: 0,...,hexadecimal,color_name,parent,size_europe,lookiero,uk,"""order""",visual_order,feature_type_name,feature_value_name
0,e652ced0-571b-4ebe-a349-47155e190141,1281248,6183657f-3a82-449d-aa6a-a14e96b0e9dd,317707,fa14ec83-b053-466e-afc0-ac164b77750f,9,0190622f-f028-42a8-b50e-ce310e8a9700,0,t,NaN,...,000000,black,\N,"{""size"": ""36"", ""format"": ""36""}","{""size"": ""S"", ""format"": ""S""}","{""size"": ""8"", ""format"": ""8""}",24,11,NaN,NaN
1,bfafcfe8-d079-49dc-b1b1-a2aabbe5c37f,1094633,6183657f-3a82-449d-aa6a-a14e96b0e9dd,\N,43482ada-0c6d-424d-8b89-bd908e8f9019,9,91636046-2652-48e4-bf82-5409418cef04,0,t,NaN,...,000000,black,\N,"{""size"": ""44"", ""format"": ""44""}","{""size"": ""XXL"", ""format"": ""XXL""}","{""size"": ""16"", ""format"": ""16""}",85,31,NaN,NaN
2,76e76793-a12b-4301-a885-ef20c8221b4c,1362329,a7c9a275-6120-4b08-9d3b-d5bad738b779,\N,c1819463-a16d-4b4c-ad1b-69f72d8fa5e6,9,77df887d-f7da-42bd-9ebf-a5a2ca57974f,0,t,NaN,...,d0455f,strawberry,65817ecb-41bf-4018-8f1f-5058aa6bc220,"{""size"": ""38"", ""format"": ""38""}","{""size"": ""M"", ""format"": ""M""}","{""size"": ""10"", ""format"": ""10""}",38,16,NaN,NaN
3,ee82ca9d-298d-4d09-a31f-6fe7b2de2ea5,1018035,c0594af7-052c-4489-9b37-d7c36eb16d39,\N,41784eb7-c52d-463d-96a8-edbd552ee0ad,8,91636046-2652-48e4-bf82-5409418cef04,0,t,NaN,...,C19A6B,camel,8c400bae-e7bf-48a0-a5bb-d2dcdc69a137,"{""size"": ""44"", ""format"": ""44""}","{""size"": ""XXL"", ""format"": ""XXL""}","{""size"": ""16"", ""format"": ""16""}",85,31,NaN,NaN
4,b33449ba-5705-4f39-ba8a-d5ed28924914,848336,f47177d1-7e0e-4b11-bacc-41d3f53b0b3d,71140,623b22c3-871e-42cb-9256-3b4f93332e11,7,77df887d-f7da-42bd-9ebf-a5a2ca57974f,0,f,NaN,...,FFFFFF,white,\N,"{""size"": ""38"", ""format"": ""38""}","{""size"": ""M"", ""format"": ""M""}","{""size"": ""10"", ""format"": ""10""}",38,16,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661998,349a4aeb-d8ca-4f3d-9efb-5b333da22290,1035346,b67d3a44-549f-4cf9-baab-a3b1090b0103,319676,92fc6853-50ab-4bda-ab2e-5a60d1e00a43,9,4b35a2af-df87-480f-a4a7-2c57f16fbdf3,63,t,2418.0,...,00008b,blue_dark,b259f6bb-e395-41d3-922a-97b5bbca3631,"{""size"": ""34"", ""format"": ""34""}","{""size"": ""XS"", ""format"": ""XS""}","{""size"": ""6"", ""format"": ""6""}",12,6,sleeve_long_cm,45
661999,349a4aeb-d8ca-4f3d-9efb-5b333da22290,1035346,b67d3a44-549f-4cf9-baab-a3b1090b0103,319676,92fc6853-50ab-4bda-ab2e-5a60d1e00a43,9,4b35a2af-df87-480f-a4a7-2c57f16fbdf3,63,t,2418.0,...,00008b,blue_dark,b259f6bb-e395-41d3-922a-97b5bbca3631,"{""size"": ""34"", ""format"": ""34""}","{""size"": ""XS"", ""format"": ""XS""}","{""size"": ""6"", ""format"": ""6""}",12,6,long_cm,65
662000,349a4aeb-d8ca-4f3d-9efb-5b333da22290,1035346,b67d3a44-549f-4cf9-baab-a3b1090b0103,319676,92fc6853-50ab-4bda-ab2e-5a60d1e00a43,9,4b35a2af-df87-480f-a4a7-2c57f16fbdf3,63,t,2418.0,...,00008b,blue_dark,b259f6bb-e395-41d3-922a-97b5bbca3631,"{""size"": ""34"", ""format"": ""34""}","{""size"": ""XS"", ""format"": ""XS""}","{""size"": ""6"", ""format"": ""6""}",12,6,back_neckline,true
662001,349a4aeb-d8ca-4f3d-9efb-5b333da22290,1035346,b67d3a44-549f-4cf9-baab-a3b1090b0103,319676,92fc6853-50ab-4bda-ab2e-5a60d1e00a43,9,4b35a2af-df87-480f-a4a7-2c57f16fbdf3,63,t,2418.0,...,00008b,blue_dark,b259f6bb-e395-41d3-922a-97b5bbca3631,"{""size"": ""34"", ""format"": ""34""}","{""size"": ""XS"", ""format"": ""XS""}","{""size"": ""6"", ""format"": ""6""}",12,6,gripper,false


## Creamos csv con caracteristicas por columna

In [13]:
columnas_fijas = [
    'variant_id', 
    'barcode', 
    'color_id', 
    'color_name', 
    'product_id', 
    'product_name',  # Asegúrate de que esta columna existe en df_final_big
    'legacy_id_x', 
    'season_code', 
    'family_id',
    'size_id', 
    'stock', 
    'publishable', 
    'hexadecimal', 
    'parent', 
    'size_europe', 
    'lookiero', 
    'uk', 
    '"order"', 
    'visual_order'
]

# 2. Creamos el nuevo DataFrame pivotando la tabla
df_features = df_final_big.pivot_table(
    index=columnas_fijas,
    columns='feature_type_name',
    values='feature_value_name',
    aggfunc='first'
).reset_index()




In [14]:
# 3. Limpieza: Eliminar el nombre de la jerarquía
df_features.columns.name = None
df_features = df_features.sort_values(by=['product_name'])
df_features

,variant_id,barcode,color_id,color_name,product_id,product_name,legacy_id_x,season_code,family_id,size_id,...,skirt_type,sleeve,sleeve_long,sleeve_long_cm,style,thicknees,top_type,type_of_length,waist_contour,weather
14545,9d7d2289-d433-4953-9293-bd6a367ad821,973236,6183657f-3a82-449d-aa6a-a14e96b0e9dd,black,1017ab52-2857-4b8f-8e07-c7bd6db0cd38,Abba Top miniprint,286879,8,39792668-5cfd-42c2-8872-ad776e9bad70,cbbdcb38-00c1-4ec2-a76b-cdf948cc81ce,...,NaN,basic_sleeve,short_sleeve,21,boho,NaN,tops,NaN,57,warm
15564,a8bdd00b-1602-48d4-82d4-fde9b38b87fe,973274,6183657f-3a82-449d-aa6a-a14e96b0e9dd,black,1017ab52-2857-4b8f-8e07-c7bd6db0cd38,Abba Top miniprint,286880,8,39792668-5cfd-42c2-8872-ad776e9bad70,2a00b162-01e4-4b28-a37e-78bcd62bf792,...,NaN,basic_sleeve,short_sleeve,21,boho,NaN,tops,NaN,57,warm
9649,6750c090-8ce3-4b67-a1fe-7ebfd313a325,973250,6183657f-3a82-449d-aa6a-a14e96b0e9dd,black,1017ab52-2857-4b8f-8e07-c7bd6db0cd38,Abba Top miniprint,286882,8,39792668-5cfd-42c2-8872-ad776e9bad70,91636046-2652-48e4-bf82-5409418cef04,...,NaN,basic_sleeve,short_sleeve,21,boho,NaN,tops,NaN,57,warm
15701,aa5ff416-c555-4886-b381-fe67a8054ce6,973267,6183657f-3a82-449d-aa6a-a14e96b0e9dd,black,1017ab52-2857-4b8f-8e07-c7bd6db0cd38,Abba Top miniprint,286883,8,39792668-5cfd-42c2-8872-ad776e9bad70,0a034f85-f04b-44ff-8770-ca2966cf2e0d,...,NaN,basic_sleeve,short_sleeve,21,boho,NaN,tops,NaN,57,warm
9207,629b6518-d1a5-4664-bc14-e8f417d9bc5e,973243,6183657f-3a82-449d-aa6a-a14e96b0e9dd,black,1017ab52-2857-4b8f-8e07-c7bd6db0cd38,Abba Top miniprint,286881,8,39792668-5cfd-42c2-8872-ad776e9bad70,a1b735a2-d6e2-4bc8-8502-ccab18802622,...,NaN,basic_sleeve,short_sleeve,21,boho,NaN,tops,NaN,57,warm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,0022ad27-6161-4cf8-a660-df6f390d7ef0,1359404,5d99a3cc-30b6-4ef8-8943-c0168adf48b1,khaki,a2f1aa36-6a3a-4394-ad9e-148227dc89c6,Zubasic Sweater hedge,\N,9,1b93b30c-833f-45a8-ba69-c21d2e7f4824,91636046-2652-48e4-bf82-5409418cef04,...,NaN,down_shoulders,3_4,39,classic,NaN,sweaters,NaN,54,cold
2668,1c95b900-4744-4e6d-992d-395548dbc049,1359367,5d99a3cc-30b6-4ef8-8943-c0168adf48b1,khaki,a2f1aa36-6a3a-4394-ad9e-148227dc89c6,Zubasic Sweater hedge,\N,9,1b93b30c-833f-45a8-ba69-c21d2e7f4824,0190622f-f028-42a8-b50e-ce310e8a9700,...,NaN,down_shoulders,3_4,39,classic,NaN,sweaters,NaN,54,cold
20150,da72ad32-d2a6-4bad-8474-15a9f45984c8,1359374,5d99a3cc-30b6-4ef8-8943-c0168adf48b1,khaki,a2f1aa36-6a3a-4394-ad9e-148227dc89c6,Zubasic Sweater hedge,\N,9,1b93b30c-833f-45a8-ba69-c21d2e7f4824,77df887d-f7da-42bd-9ebf-a5a2ca57974f,...,NaN,down_shoulders,3_4,39,classic,NaN,sweaters,NaN,54,cold
2434,19f3c912-9f84-4019-bc67-c260ed535b84,1359398,5d99a3cc-30b6-4ef8-8943-c0168adf48b1,khaki,a2f1aa36-6a3a-4394-ad9e-148227dc89c6,Zubasic Sweater hedge,\N,9,1b93b30c-833f-45a8-ba69-c21d2e7f4824,a1b735a2-d6e2-4bc8-8502-ccab18802622,...,NaN,down_shoulders,3_4,39,classic,NaN,sweaters,NaN,54,cold


### Seleccion de caracteristicas importantes para creacion del grafo

In [16]:
df_final = df_features.drop(columns=["size_europe", "lookiero", "uk", 'product_id','color_id' , 'variant_id', 'size_id', 'barcode','legacy_id_x','stock','publishable','hexadecimal','"order"','visual_order'])
df_final= df_final.drop_duplicates()
filtro= df_final[['family_id', 'product_name']].drop_duplicates()
df_final

,color_name,product_name,season_code,family_id,parent,adventurous,alerts,application,back_neckline,bag_type,...,skirt_type,sleeve,sleeve_long,sleeve_long_cm,style,thicknees,top_type,type_of_length,waist_contour,weather
14545,black,Abba Top miniprint,8,39792668-5cfd-42c2-8872-ad776e9bad70,\N,two,NaN,freetime,NaN,NaN,...,NaN,basic_sleeve,short_sleeve,21,boho,NaN,tops,NaN,57,warm
20217,blue_dark,Abba Top print,8,39792668-5cfd-42c2-8872-ad776e9bad70,b259f6bb-e395-41d3-922a-97b5bbca3631,two,NaN,freetime,NaN,NaN,...,NaN,japanese_sleeve,short_sleeve,17,boho,NaN,tops,NaN,57,warm_season
22366,black,Abbey Sweater embroidered,9,1b93b30c-833f-45a8-ba69-c21d2e7f4824,\N,two,NaN,work,NaN,NaN,...,NaN,basic_sleeve,long_sleeve,57,classic,NaN,sweaters,NaN,46,cold_season
2067,red,Abby Dress caribbean,7,ba89fcc3-bb1b-42af-a754-2d9f639e43ab,\N,three,breastfeeding,work,NaN,NaN,...,NaN,basic_sleeve,short_sleeve,23,classic,NaN,NaN,knee,40,warm
70,black,Abelone Playsuit miniprint,7,88ee8b83-a7fe-4c00-8f12-621b209012fd,\N,four,NaN,freetime,NaN,NaN,...,NaN,bare_shoulders,short_sleeve,26,boho,NaN,NaN,short,39,warm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10119,black,Zinka Shirt flower,7,c59807ac-0362-4078-94b0-37ea3df6cdd0,\N,two,breastfeeding,work,NaN,NaN,...,NaN,folded_up_with_loops,3_4,52,classic,NaN,blouse,NaN,52,warm_season
7342,blue_dark,Zipe Sweater season,7,3087da56-5a27-4e58-aea7-1555c640f232,b259f6bb-e395-41d3-922a-97b5bbca3631,three,special_size,freetime,NaN,NaN,...,NaN,down_shoulders,long_sleeve,64,casual,NaN,hoodies,NaN,69,warm_season
11552,red,Zipe Sweater season,7,3087da56-5a27-4e58-aea7-1555c640f232,\N,three,special_size,freetime,NaN,NaN,...,NaN,down_shoulders,long_sleeve,64,casual,NaN,hoodies,NaN,69,warm_season
263,gray_light,Zoe Cardigan long,8,141f32d6-a5af-49bf-b019-fd28354977f1,9a1c2e1b-a833-449e-8202-d9d664d72a04,two,NaN,freetime,NaN,NaN,...,NaN,down_shoulders,long_sleeve,70,boho,NaN,cardigan,NaN,56,cold_season


In [28]:
data = {
    'family_id': [
        '39792668-5cfd-42c2-8872-ad776e9bad70',
        '1b93b30c-833f-45a8-ba69-c21d2e7f4824',
        'ba89fcc3-bb1b-42af-a754-2d9f639e43ab',
        '88ee8b83-a7fe-4c00-8f12-621b209012fd',
        'c59807ac-0362-4078-94b0-37ea3df6cdd0',
        'c6b81db8-29ed-43d6-b14d-0fd913d7ce36',
        '9c5a3fee-dad0-444a-acb3-c059e2f3db20',
        '2514d6c8-11e6-4692-87b4-dcc1cd18592c',
        '95f835a3-03fe-40f1-a813-4fbfd2842845',
        '1dc55e58-a5c3-462c-b73d-8dd480de90fc',
        '3087da56-5a27-4e58-aea7-1555c640f232',
        '6598b655-3e19-448c-8b4f-ff6a29447b34',
        '141f32d6-a5af-49bf-b019-fd28354977f1',
        '35405a7d-7018-440b-9d16-76206ac3e224',
        '07bee010-39f3-4200-9250-c1524842e6a6',
        'b263ba41-adab-4a83-a3e0-2766522f5aec',
        '882ecdf8-325d-4826-a066-1b263e438f09',
        'e1646b68-8f10-4b63-87d5-30924341f8ba',
        '74ba5e25-afd5-490b-a3a8-2175020c4487'
    ],
    'nivel': [
        2, 2, 1, 1, 2, 3, 1, 1, 2, 
        1, 2, 1, 2, 3, 3, 3, 3, 3, 3
    ]
}

df_niveles = pd.DataFrame(data)

In [29]:
df_final = df_final.merge(df_niveles[['family_id', 'nivel']], on='family_id', how='left')
columnas = [
    "color_name",
    "product_name",
    "season_code",
    "adventurous",
    "application",
    "composition",
    "cut",
    "style",
    "weather",
    "nivel",
    "print"
]

df_filtrado = df_final[columnas]
df_filtrado.to_csv('Datos alumnos/Trasformados/df_para_grafo.csv')

# PARTE DE LOS COLORES (JUNTAMOS)

In [ ]:
df_sino = pd.read_csv('Datos alumnos/Trasformados/mis_votos_colores.csv')
df_punt = pd.read_csv('Datos alumnos/Trasformados/mis_puntuaciones_colores.csv')

In [ ]:
mask_yes = df_sino['Match'] == "YES"

# Merge de las filas YES de df_sino con df_punt para obtener el Rating correcto
df_merge = df_sino[mask_yes].merge(
    df_punt[['Color A', 'Color B', 'Rating']],
    on=['Color A', 'Color B'],
    how='left'
)

df_sino.loc[mask_yes, 'Score'] = df_merge['Rating'].values
df_sino['Score'] = df_sino['Score'].apply(lambda x: 0 if x < 4 else x)
df_sino.to_csv('Datos alumnos/Trasformados/df_sino_actualizado.csv', index=False)